# Projeto - Grupo 1 - Técnicas de Programação II

## Integrantes do Grupo: Caio Brainer, Thaisa Elvas, Alex Amaro, Luana Rodrigues, Lucindo e Henricco

Referências:

https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce#olist_products_dataset.csv [Dados de eccomerce 2016-2018 Olist]

https://www.kaggle.com/datasets/thiagobodruk/brazil-geojson [Dados JSON do Brasil e seus estados]

https://github.com/filipegmedeiros/brazilian_choropleth_map/tree/main/geojson [Dados JSON do Brasil e seus estados]

In [ ]:
import pandas as pd
import plotly.express as px
import json
import plotly.io as pio
pio.renderers.default = 'jupyterlab'

In [ ]:
# Carga inicial dos dados
df_order_payments = pd.read_csv("dados/olist_order_payments_dataset.csv")
df_order_dataset = pd.read_csv("dados/olist_orders_dataset.csv")
df_order_customer = pd.read_csv("dados/olist_customers_dataset.csv")
df_order_itens = pd.read_csv("dados/olist_order_items_dataset.csv")
df_order_products = pd.read_csv("dados/olist_products_dataset.csv")
df_order_sellers = pd.read_csv("dados/olist_sellers_dataset.csv")
df_order_geolocation = pd.read_csv("dados/olist_geolocation_dataset.csv")

# Geojson
geojson = json.load(open("geojson/brasil_estados.json"))

In [ ]:
# Merge dos dados
df_order_geolocation.rename(columns={'geolocation_zip_code_prefix' : 'zip_code_prefix'}, inplace=True)
df_unido = pd.merge(df_order_payments, df_order_dataset, how="inner", on="order_id")
df_unido = pd.merge(df_unido, df_order_itens, how="inner", on="order_id")
df_unido = pd.merge(df_unido, df_order_customer, how="inner", on="customer_id")
df_unido = pd.merge(df_unido, df_order_products, how="inner", on="product_id")
df_unido = pd.merge(df_unido, df_order_sellers, how="inner", on="seller_id")
df_unido.rename(columns={'customer_zip_code_prefix' : 'zip_code_prefix'}, inplace=True)

In [ ]:
df_unido_amostra = df_unido.sample(frac=0.2, random_state=42)
df_unido_amostra.dropna(inplace=True)
#df_unido_amostra.to_parquet("dados/amostra.parquet")

## Código Luana

In [ ]:
# Código da Luana
clientes_por_estado = df_unido_amostra.groupby('customer_state')['customer_id'].nunique().sort_values(
    ascending=False).reset_index()
media_frete_por_estado = df_unido_amostra.groupby('customer_state')['freight_value'].mean().sort_values(
    ascending=False).reset_index()
vendedores_por_estado = df_unido_amostra.groupby('seller_state')['seller_id'].nunique().sort_values(
    ascending=False).reset_index()

merged_df = clientes_por_estado.merge(media_frete_por_estado, on='customer_state')
merged_df.rename(columns={'customer_id': 'clientes_unicos', 'freight_value': 'valor_medio_frete', 'seller_id': 'num_vendedores'}, inplace=True)
merged_df = merged_df.merge(vendedores_por_estado, 
                            how='outer', left_on='customer_state', 
                            right_on='seller_state').sort_values(by='clientes_unicos', ascending=False)

merged_df.rename(columns={'customer_id': 'clientes_unicos', 'freight_value': 'valor_medio_frete', 'seller_id': 'num_vendedores'}, inplace=True)
merged_df = merged_df.sort_values(by='clientes_unicos', ascending=False)

## Código da Thaisa

In [ ]:
# Código da Thaisa
df_unido_amostra['order_approved_at'] = pd.to_datetime(df_unido_amostra['order_approved_at']).dt.date
df_unido_amostra['order_delivered_customer_date'] = pd.to_datetime(df_unido_amostra['order_delivered_customer_date']).dt.date
df_unido_amostra['order_estimated_delivery_date'] = pd.to_datetime(df_unido_amostra['order_estimated_delivery_date']).dt.date

df_unido_amostra['tempo_de_entrega'] = (pd.to_timedelta(df_unido_amostra['order_delivered_customer_date'] -
                                                       df_unido_amostra['order_approved_at']).dt.days).astype(int)

df_unido_amostra['tempo_estimado'] = (pd.to_timedelta(df_unido_amostra['order_estimated_delivery_date'] -
                                                     df_unido_amostra['order_approved_at']).dt.days).astype(int)


tempo_de_entrega = df_unido_amostra[['customer_state', 'tempo_de_entrega', 'tempo_estimado']]
tempo_de_entrega['dias'] = df_unido_amostra['tempo_de_entrega'] - df_unido_amostra['tempo_estimado']
df_atrasados = tempo_de_entrega[tempo_de_entrega['dias'] > 0].groupby('customer_state').size().reset_index().sort_values(by='customer_state')

# Tabelas temporárias para análise e gráfico de barras com os estados que apresentam maior proporção de atrasos 
df_qtd = tempo_de_entrega.groupby('customer_state').size().reset_index().sort_values(by='customer_state')
df_unido = pd.merge(df_qtd, df_atrasados, how='inner', on='customer_state')
df_unido['prop'] = df_unido['0_y'] / df_unido['0_x']
df_unido.sort_values(by='prop', ascending=False)

px.bar(df_unido.sort_values(by='prop', ascending=False).head(), x='customer_state', y='prop')

In [ ]:
# Tratamento para o boxplot com a distribuição de dias de atraso nos estados com maiores proporções de atraso
filtro_estados = (tempo_de_entrega['dias'] > 0) & (tempo_de_entrega['customer_state'].isin(['MA', 'TO', 'AL', 'SE', 'PI']))
df_atrasos = tempo_de_entrega[filtro_estados]
fig = px.box(df_atrasos, x="customer_state", y="dias")
fig.show()

## Código Lucindo

In [ ]:
## Média de vendas para o Dias das Mães de 2017 (por dia)

# Suponha que você tenha um DataFrame df com as colunas 'order_purchase_timestamp' e 'price'
# Converta a coluna 'order_purchase_timestamp' para o tipo datetime
df_unido_amostra['order_purchase_timestamp'] = pd.to_datetime(df_unido_amostra['order_purchase_timestamp'])

# Defina a data alvo e o período de 10 dias anteriores
data_alvo = pd.to_datetime('2017-05-14')
data_inicial = data_alvo - pd.Timedelta(days=10)

# Filtrar as linhas correspondentes ao período de 10 dias anteriores à data alvo
df_filtrado = df_unido_amostra[(df_unido_amostra['order_purchase_timestamp'] >= data_inicial) & (df_unido_amostra['order_purchase_timestamp'] <= data_alvo)]

# Agrupar vendas pela coluna 'order_purchase_timestamp' e calcular o valor médio na coluna 'price'
vendas_agrupadas17 = df_filtrado.groupby(df_filtrado['order_purchase_timestamp'].dt.date)['price'].mean()

## Média de vendas para o Dias das Mães de 2018 (por dia)

# Converta a coluna 'order_purchase_timestamp' para o tipo datetime
df_unido_amostra['order_purchase_timestamp'] = pd.to_datetime(df_unido_amostra['order_purchase_timestamp'])

# Defina a data alvo e o período de 10 dias anteriores
data_alvo = pd.to_datetime('2018-05-13')
data_inicial = data_alvo - pd.Timedelta(days=10)

# Filtrar as linhas correspondentes ao período de 10 dias anteriores à data alvo
df_filtrado = df_unido_amostra[(df_unido_amostra['order_purchase_timestamp'] >= data_inicial) & (df_unido_amostra['order_purchase_timestamp'] <= data_alvo)]

# Agrupar vendas pela coluna 'order_purchase_timestamp' e calcular o valor médio na coluna 'price'
vendas_agrupadas18 = df_filtrado.groupby(df_filtrado['order_purchase_timestamp'].dt.date)['price'].mean()


In [ ]:
dias = pd.to_datetime(vendas_agrupadas18.reset_index()['order_purchase_timestamp'])
dias.dt.day